#공행성 쌍 찾기
공행성이라는 건 인과관계를 의미하는 게 아니라 서로 같이 움직이는 물건을 찾는 것 즉 A가 떨어질 때 B가 떨어지고 그게 지속적으로 나타나면 그건 공행성 쌍이라고 취급할 것
근데 이때 생각해야할 것:
과연 몇번 같이 움직인 걸 공행성 쌍으로 볼 것인가?
몇 개월을 사이에 두고 움직인 걸 공행성 쌍으로 볼 것인가?
같이 행동하는 것처럼 보여도 완벽하다고는 볼 수 없어서 퍼센트 비율을 잘 줘야될 듯
같이 움직였다라는 걸 어떤 기준으로 볼 것인가?

##하고자 하는 것
A 품목이 변할 때 일정 시차를 두고 B 품목도 비슷하게 변하면 (일정 시차를 어떻게 할 건데?)
(A, B)는 공행성 쌍
이 관계를 정확히 찾는 게 1단계
###공행성 쌍을 최대한 정확하게 찾는 방법
공행성은 절댓값보다 증감률로 파악해야 함


공행성에 대한 정의 및 탐색 기준
1. 같이 움직였다의 기준은? (측정지표)
로그 변환후 차분: 월별 증감률을 사용. 이는 np.log1-를 적용한 뒤 .diff(1)을 통해 얻음. 이 값은 정상성을 확보하고 백분율 변화와 유사하게 해석할 수 있어서 시계열 상관 분석에 가장 적합.
피어슨 상관계수 : 두 품목의 로그 차분 값 간의 피어슨 상관계수를 사용
"어느 정도 같이 움직여야 하는가?" (임계값)
1차 기준 (상관계수) : abs(상관계수 ) > 0.6 (높은 상관관계) - 상관관계 값을 바꿔가며 성능 확인해볼 수 있음
2차 기준 (안정성) : 지속적인 관계여야 함. -12개월 롤링 상관계수의 표준편차가 낮아야 합니다. (예 : 0.2 미만)
시차는 얼마나 인정할 것인가?
6개월까지??
따라서 1<= lag <= 3인 쌍을 최종후보로 우선할 거임 0개월 시차는 예측에 사용할 수 없고 4개월 이상은 관계가 약해질 가능성이 있

In [ ]:
#모든 품목의 월별 변화율
import pandas as pd
import numpy as np
from itertools import combinations

# 1. 데이터 로드
train = pd.read_csv('train.csv')

# 2. 날짜 생성 및 월별/HS4별 집계
train['date'] = pd.to_datetime(train[['year', 'month']].assign(day=1))
monthly_data = train.groupby(['date', 'hs4'])['value'].sum().reset_index()

# 3. 피벗 테이블 생성 (index=date, columns=hs4, values=value)
#    fillna(0)으로 해당 월에 거래가 없었음을 명시
pivot_table = monthly_data.pivot(index='date', columns='hs4', values='value').fillna(0)

print(f"피벗 테이블 생성 완료: {pivot_table.shape}")

피벗 테이블 생성 완료: (43, 71)


In [ ]:
#상관 기반 1차 공행성 후보 탐색
#1차는 단순하게 같이 움직이는 품목 후보를 빠르게 찾을거야

 #상관계수가 0.7이상인 품목쌍만 우선 후보로 추출, 이때는 시차 고려 X
#이렇게 하면 전체 품목 중에서 함께 움직이는 경향이 강한 쌍만 남음
#근데 문제는 시차를 두고 움직이는 물건은 아예 고려하지 못함

In [ ]:
#후보 쌍에 대해 시간차를 바꿔가며 상관계수를 계산
# 4. 분석 대상 컬럼 필터링
# 4-1. 거래가 너무 희소한 품목 제외 (예: 전체 기간의 75% 이상 0인 품목)
min_obs = len(pivot_table) * 0.25
sparse_cols = pivot_table.columns[pivot_table.astype(bool).sum(axis=0) < min_obs]
pivot_filtered = pivot_table.drop(columns=sparse_cols)

# 4-2. 저변동성 품목 제외 (사용자 아이디어)
std = pivot_filtered.std()
low_std_cols = std[std < std.quantile(0.20)].index
pivot_filtered = pivot_filtered.drop(columns=low_std_cols)

# 5. 시계열 변환 (로그 변환 -> 1차 차분)
# 이것이 '월별 변화율'을 나타냄
pivot_diff = pivot_filtered.apply(lambda x: np.log1p(x)).diff().dropna()

print(f"최종 분석 대상 HS4 품목 수: {len(pivot_diff.columns)}")

#lag > 0 -> A가 선행, B가 후행
#lag < 0 -> B가 선행, A가 후행
#단 corr > 0.6이상 lag <- 3 정도만 채택 (시차 너무 길면 약함)




최종 분석 대상 HS4 품목 수: 51


In [ ]:
# 사용자가 제안한 최적 시차 탐색 함수 (수정 없음)
def find_best_lag(a, b, max_lag=6):
    best_corr, best_lag = 0, 0
    for lag in range(-max_lag, max_lag + 1):
        if lag < 0:
            # B가 A를 lag만큼 선행 (A가 후행)
            corr = np.corrcoef(a[abs(lag):], b[:lag])[0, 1]
        elif lag > 0:
            # A가 B를 lag만큼 선행 (B가 후행)
            corr = np.corrcoef(a[:-lag], b[lag:])[0, 1]
        else:
            # 동행
            corr = np.corrcoef(a, b)[0, 1]

        if abs(corr) > abs(best_corr):
            best_corr, best_lag = corr, lag
    return best_corr, best_lag

# 6. 공행성 쌍 탐색
hs4_codes = pivot_diff.columns
comovement_pairs = []

# 롤링 윈도우 크기 (1년)
ROLLING_WINDOW = 12

# 임계값 설정
MIN_CORR = 0.5        # 최소 상관계수 최소 상관계수가 0.6이면 만족하는 쌍이 없음
MAX_LAG = 3           # 최대 허용 시차 (A가 선행)
MIN_LAG = 1           # 최소 허용 시차 (A가 선행) -> 최소 허용 시차가 1인 이유는 단순히 쌍을 찾는 게 아니라 선행, 후행이 존재하기 때문
MAX_STABILITY_STD = 0.2 # 롤링 상관계수의 최대 표준편차 (낮을수록 안정적)

print(f"탐색 시작: {len(hs4_codes)}개 품목 기준 약 {len(hs4_codes)*(len(hs4_codes)-1)//2}개 쌍 탐색...")

for (col_a, col_b) in combinations(hs4_codes, 2):
    series_a = pivot_diff[col_a]
    series_b = pivot_diff[col_b]

    # 1. 최적 시차 및 상관계수 탐색
    best_corr, best_lag = find_best_lag(series_a, series_b)

    # 2. (A -> B) 선후행 관계 필터링 (A가 선행, B가 후행)
    # lag > 0 (A가 B를 선행) 이고, 상관계수 임계값 통과
    if MIN_LAG <= best_lag <= MAX_LAG and best_corr > MIN_CORR:
        # 3. 안정성 검증
        # A(t)와 B(t + lag)의 관계를 확인해야 함
        # b.shift(-best_lag)는 B의 데이터를 best_lag만큼 앞으로 당겨옴
        rolling_corr = series_a.rolling(window=ROLLING_WINDOW).corr(series_b.shift(-best_lag))

        stability_std = rolling_corr.std()

        if stability_std < MAX_STABILITY_STD:
            comovement_pairs.append({
                'A_hs4': col_a,
                'B_hs4': col_b,
                'lag': best_lag,
                'corr': best_corr,
                'stability_std': stability_std
            })

    # 3. (B -> A) 선후행 관계 필터링 (B가 선행, A가 후행)
    # lag < 0 (B가 A를 선행) 이고, 상관계수 임계값 통과
    elif MIN_LAG <= abs(best_lag) <= MAX_LAG and best_corr > MIN_CORR:
        # B가 A를 abs(best_lag)만큼 선행
        lag_ba = abs(best_lag)

        # 3. 안정성 검증
        rolling_corr = series_b.rolling(window=ROLLING_WINDOW).corr(series_a.shift(-lag_ba))
        stability_std = rolling_corr.std()

        if stability_std < MAX_STABILITY_STD:
            comovement_pairs.append({
                'A_hs4': col_b, # B가 선행 품목(A)
                'B_hs4': col_a, # A가 후행 품목(B)
                'lag': lag_ba,
                'corr': best_corr,
                'stability_std': stability_std
            })

print("탐색 완료.")

# 4. 결과 정렬 및 확인
if comovement_pairs:
    result_df = pd.DataFrame(comovement_pairs)
    result_df = result_df.sort_values(by='corr', ascending=False)
    print(result_df.head(10))
else:
    print("기준을 만족하는 공행성 쌍을 찾지 못했습니다.")

탐색 시작: 51개 품목 기준 약 1275개 쌍 탐색...
탐색 완료.
   A_hs4  B_hs4  lag      corr  stability_std
5   3404   8479    2  0.587013       0.160257
1   2805   3006    2  0.555913       0.162290
6   3904   6101    3  0.555441       0.072161
4   3006   2846    1  0.543421       0.127561
3   2847   2841    3  0.542756       0.186104
2   7202   2836    2  0.534239       0.090463
0   2529   4408    3  0.530792       0.171221
7   6211   7202    3  0.505911       0.128517


##의문점
찾아보니 첫번째로 나온 쌍이 인조왁스, 조제왁스랑 토목 건축용 기계 모올타르 살포기 이게 맞다면 건설 경기나 프로젝트 사이클이 커질 때 두품목이 동시에 증가하는 공행성이 생긴 것
즉 거시적 공행성이라는 뜻 두 물품사이에 직접적인 연관은 없는데 경기 때문에 둘이 같이 증가한 것일 수 있음
그렇다면 이러한 거시적 공행성은 넣지 않는 게 맞다고 생각

##가짜 공행성을 걸러내기 위해 고려할 점
### 잠재적 공통 요인(모든 품목에 공통으로 작용하는 계절성과 시장 전체의 움직임을 최대한 제거하는 게 좋음)
관계의 일관성/안전정
롤링 상관계수의 표준편차를 보는 것에 더해 롤링 상관계수의 평균도 일정 수준인지 확인해야 함. 관계가 꾸준히 + 같이 변화하는 방향으로 유지
도메인 지식을 배제해야할지 확신을 못하겠음 : hs 코드를 직접 비교해가면서 서로 연관 있는 제품이 아니면 다시 공행성 쌍을 구하는 방식으로 하고 있는데 과연 hs코드가 확실히 그 물품을 말해주는 것도 아니라 고민
임계값 설정 : MIN_CORR = 0.6에서 0.5로 낮추니 가짜가 많이 나왔습니다. 단순히 상관관계수만 높이는 것은 해결책이 아니야/ 안정성 기준이 매우 엄격해야 함

##수정사항
(vs. 공통 요인) ➡️ 계절성 요인 제거: pivot_diff(월별 증감률)에서 월별 평균 증감률을 빼주어, 계절적 요인을 제거한 '순수 변동분'으로 상관관계를 계산합니다.

(vs. 일관성) ➡️ 안정성 기준 강화:

MAX_STABILITY_STD (변동성) 기준을 0.2에서 **0.15**로 강화합니다.

MIN_STABILITY_MEAN (평균 일관성) 기준 **0.3**을 신규 추가합니다. (전체 상관계수가 0.5여도, 12개월 롤링 평균이 최소 0.3은 꾸준히 나와야 함)

(vs. 도메인) ➡️ HS코드명 매핑: 최종 후보 쌍(result_df)에 관세청_HS부호_Data.csv 파일을 읽어와 한글 품목명을 매칭하여, 결과 해석을 용이하게 합니다.


In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

train = pd.read_csv('train.csv')
#HS 코드 데이터 로드
hs_code_df = pd.read_csv('관세청_HS부호_20240101.csv')

#2. HS코드 맵 생성 (HS4 기준)
#HS 부호를 10자리로 맞추고 앞 4자리 추출
hs_code_df['hs4'] = hs_code_df['HS부호'].astype(str).str.pad(10, 'left', '0').str.slice(0, 4)

#hs4 코드별로 대표 품목명(한글) 맵 생성
hs4_map = hs_code_df.drop_duplicates(subset=['hs4']).set_index('hs4')['한글품목명'].to_dict()
print("HS코드 맵 생성 완료.")

# 3. 날짜 생성 및 월별/HS4별 집계
train['date'] = pd.to_datetime(train[['year', 'month']].assign(day=1))
monthly_data = train.groupby(['date', 'hs4'])['value'].sum().reset_index()

# 4. 피벗 테이블 생성
pivot_table = monthly_data.pivot(index='date', columns='hs4', values='value').fillna(0)

# 5. 분석 대상 컬럼 필터링 (기존과 동일)
min_obs = len(pivot_table) * 0.25
sparse_cols = pivot_table.columns[pivot_table.astype(bool).sum(axis=0) < min_obs]
pivot_filtered = pivot_table.drop(columns=sparse_cols)

std = pivot_filtered.std()
low_std_cols = std[std < std.quantile(0.20)].index
pivot_filtered = pivot_filtered.drop(columns=low_std_cols)

# 6. 시계열 변환 (로그 변환 -> 1차 차분)
pivot_diff = pivot_filtered.apply(lambda x: np.log1p(x)).diff().dropna()

# 7. [신규] 계절성 제거 (De-seasonalize)
# 각 월(1월, 2월...)의 평균 증감률을 계산
monthly_avg_diff = pivot_diff.groupby(pivot_diff.index.month).transform('mean')
# 원본 증감률에서 월별 평균 증감률을 빼서 계절성을 제거
pivot_deseasoned = pivot_diff - monthly_avg_diff
print("계절성 제거 완료.")

# 최적 시차 탐색 함수 (기존과 동일)
def find_best_lag(a, b, max_lag=6):
    best_corr, best_lag = 0, 0
    for lag in range(-max_lag, max_lag + 1):
        if lag < 0:
            corr = np.corrcoef(a[abs(lag):], b[:lag])[0, 1]
        elif lag > 0:
            corr = np.corrcoef(a[:-lag], b[lag:])[0, 1]
        else:
            corr = np.corrcoef(a, b)[0, 1]

        if abs(corr) > abs(best_corr):
            best_corr, best_lag = corr, lag
    return best_corr, best_lag

# 8. 공행성 쌍 탐색 (기준 강화)
hs4_codes = pivot_deseasoned.columns
comovement_pairs = []

ROLLING_WINDOW = 12

# 임계값 재설정
MIN_CORR = 0.5            # 최소 상관계수 (유지)
MAX_LAG = 3               # 최대 허용 시차
MIN_LAG = 0.5               # 최소 허용 시차 (선후행)
MAX_STABILITY_STD = 0.15  # [기준 강화] 롤링 상관계수 표준편차 (0.2 -> 0.15)
MIN_STABILITY_MEAN = 0.3  # [기준 추가] 롤링 상관계수 평균 (꾸준히 0.3 이상)

print(f"탐색 시작: {len(hs4_codes)}개 품목, 계절성 제거 데이터 기준...")

for (col_a, col_b) in combinations(hs4_codes, 2):
    # 계절성이 제거된 데이터로 분석
    series_a = pivot_deseasoned[col_a]
    series_b = pivot_deseasoned[col_b]

    best_corr, best_lag = find_best_lag(series_a, series_b)

    # (A -> B) 선후행 관계
    if MIN_LAG <= best_lag <= MAX_LAG and abs(best_corr) > MIN_CORR:
        rolling_corr = series_a.rolling(window=ROLLING_WINDOW).corr(series_b.shift(-best_lag))
        stability_std = rolling_corr.std()
        stability_mean = rolling_corr.mean()

        if stability_std < MAX_STABILITY_STD and stability_mean > MIN_STABILITY_MEAN:
            comovement_pairs.append({
                'A_hs4': col_a, 'B_hs4': col_b, 'lag': best_lag,
                'corr': best_corr, 'stability_std': stability_std, 'stability_mean': stability_mean
            })

    # (B -> A) 선후행 관계
    elif MIN_LAG <= abs(best_lag) <= MAX_LAG and best_corr > MIN_CORR:
        lag_ba = abs(best_lag)
        rolling_corr = series_b.rolling(window=ROLLING_WINDOW).corr(series_a.shift(-lag_ba))
        stability_std = rolling_corr.std()
        stability_mean = rolling_corr.mean()

        if stability_std < MAX_STABILITY_STD and stability_mean > MIN_STABILITY_MEAN:
            comovement_pairs.append({
                'A_hs4': col_b, 'B_hs4': col_a, 'lag': lag_ba,
                'corr': best_corr, 'stability_std': stability_std, 'stability_mean': stability_mean
            })

print("탐색 완료.")

# 9. [신규] 결과 정렬 및 HS코드명 매핑
if comovement_pairs:
    result_df = pd.DataFrame(comovement_pairs)
    result_df = result_df.sort_values(by='corr', ascending=False)

    # HS코드명 매핑
    result_df['A_item'] = result_df['A_hs4'].map(hs4_map).fillna('알 수 없음')
    result_df['B_item'] = result_df['B_hs4'].map(hs4_map).fillna('알 수 없음')

    final_columns = ['A_hs4', 'A_item', 'B_hs4', 'B_item', 'lag', 'corr', 'stability_std', 'stability_mean']
    print("\n--- 최종 공행성 후보 쌍 (계절성 제거, 안정성 기준 강화) ---")
    print(result_df[final_columns].head(20))
else:
    print("강화된 기준을 만족하는 공행성 쌍을 찾지 못했습니다.")


FileNotFoundError: [Errno 2] No such file or directory: '관세청_HS부호_20240101.csv'

좀 더 엄격하게 관리한 다음에 나온 결과는
1. 탄산이나트륨, 탄산염, 과산화탄산염    -   페로망간(탄소 2 % 초과)
2. 범퍼, 완충기 그 부분품 (차량용)     -     라디오카세트플레이어(포켓사이즈형)  ? - 범퍼는 현대 자동차 산업의 핵심 부품이지만 라디오카세트는 수요가 거의 없음. 이게 왜?
3. 요소 (광물성 비료 또는 화학비료)  -   수영복(직물제 - 인조섬유, 직물제)
4. 라디오카세트플레이어 (포켓사이즈형)  -    로진, 수지산  -> 로진은 전자기기 회로기판에 부품을 납땜할 때 쓰는 솔더 플럭스의 핵심 원료
5. 홉 (잘게 부순 것, 가루 모양, 신선.건조)    -    모직물 (양모, 동물털, 양모직물) - 농업기반 소비재 원료
6.
7. 전동기 (발전세트) - 탄산이나트륨
8. 아라미드 사, 나일론  (재봉사 소매용 제외)   -  장석, 기타 비금속광뭄ㄹ
9. 캣커트, 접착제, 라미나리아 텐트 봉합재 지혈제 유착방지제 외과용 치과용 기타의료위생용품 - 세륨 화합물 (기타정밀화학원료)  -  접점이 없
10. 탄산이나트륨(기타정밀화학원료)   -   코발트 중간생산물 (기타비철금속제품)
11. 아마직물   -페로망간

##고쳐야 할점 : 지금은 양의 상관관계만 찾고 있었음 하지만 실제로 같이 움직이기만 하면 되는 거지 같이 증가할 필요는 없음 즉 음의 상관관계까지 포함해야함

음의 상관관계 포함 (abs(best_corr)):

if best_corr > MIN_CORR (0.5보다 큰 양수만) ➡️ if abs(best_corr) > MIN_CORR (0.5보다 크거나 -0.5보다 작은, 즉 강한 관계)로 수정했습니다.

안정성 검증 수정 (abs(stability_mean)):

음의 상관관계(예: -0.6)는 롤링 평균(stability_mean)도 음수가 됩니다.

stability_mean > 0.3 조건은 모든 음의 관계를 탈락시킵니다.

따라서 abs(stability_mean) > MIN_STABILITY_MEAN (롤링 평균의 절댓값이 0.3 이상)으로 수정하여, 꾸준히 양(+)이든 꾸준히 음(-)이든 강한 관계를 유지하면 통과하도록 했습니다.

MIN_LAG 수정:

MIN_LAG = 0.5는 lag가 정수(1, 2, 3...)이므로 작동하지 않습니다. 예측을 위한 최소 시차인 MIN_LAG = 1로 다시 수정했습니다.

결과 정렬 수정:

강한 음의 상관관계(예: -0.7)가 맨 아래에 표시되는 것을 막기 위해, 상관계수 절댓값(abs_corr) 기준으로 정렬하도록 수정했습니다.

파일 로딩 오류 수정:

이전 대화에서 발생했던 파일명 및 인코딩 오류를 미리 수정하여 코드가 바로 실행되도록 했습니다. (7번째 줄)

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from lightgbm import LGBMRegressor
import warnings

warnings.filterwarnings('ignore')

train = pd.read_csv('train.csv')

# HS 코드 데이터 로드
# [수정] 이전 오류 방지를 위해 정확한 파일명과 인코딩(utf-8-sig) 적용
# hs_code_df = pd.read_csv('관세청_HS부호_20240101.csv', encoding='utf-8-sig')

# # 2. HS코드 맵 생성 (HS4 기준)
# hs_code_df['hs4'] = hs_code_df['HS부호'].astype(str).str.pad(10, 'left', '0').str.slice(0, 4)
# hs4_map = hs_code_df.drop_duplicates(subset=['hs4']).set_index('hs4')['한글품목명'].to_dict()
# print("HS코드 맵 생성 완료.")

# 3. 날짜 생성 및 월별/HS4별 집계
train['date'] = pd.to_datetime(train[['year', 'month']].assign(day=1))
monthly_data = train.groupby(['date', 'hs4'])['value'].sum().reset_index()

# 4. 피벗 테이블 생성
pivot_raw = monthly_data.pivot(index='date', columns='hs4', values='value').fillna(0)

# 5. 분석 대상 컬럼 필터링
min_obs = len(pivot_raw) * 0.25
sparse_cols = pivot_raw.columns[pivot_raw.astype(bool).sum(axis=0) < min_obs]
pivot_filtered = pivot_raw.drop(columns=sparse_cols)

std = pivot_filtered.std()
low_std_cols = std[std < std.quantile(0.20)].index
pivot_filtered = pivot_filtered.drop(columns=low_std_cols)

# 6. 시계열 변환 (로그 변환 -> 1차 차분)
pivot_diff = pivot_filtered.apply(lambda x: np.log1p(x)).diff().dropna()

# 7. 계절성 제거
monthly_avg_diff = pivot_diff.groupby(pivot_diff.index.month).transform('mean')
pivot_deseasoned = pivot_diff - monthly_avg_diff
print("계절성 제거 완료.")

# 최적 시차 탐색 함수 (기존과 동일)
def find_best_lag(a, b, max_lag=6):
    best_corr, best_lag = 0, 0
    for lag in range(-max_lag, max_lag + 1):
        if lag < 0:
            corr = np.corrcoef(a[abs(lag):], b[:lag])[0, 1]
        elif lag > 0:
            corr = np.corrcoef(a[:-lag], b[lag:])[0, 1]
        else:
            corr = np.corrcoef(a, b)[0, 1]

        if abs(corr) > abs(best_corr):
            best_corr, best_lag = corr, lag
    return best_corr, best_lag

# 8. 공행성 쌍 탐색 (기준 강화)
hs4_codes = pivot_deseasoned.columns
comovement_pairs = []

ROLLING_WINDOW = 12

# 임계값 재설정
MIN_CORR = 0.5            # 최소 상관계수 (절댓값 기준 0.5)
MAX_LAG = 3               # 최대 허용 시차
MIN_LAG = 1               # [수정] 최소 허용 시차 (0.5 -> 1, 정수)
MAX_STABILITY_STD = 0.15
MIN_STABILITY_MEAN = 0.3  # [수정] (절댓값 기준 0.3)

print(f"탐색 시작: {len(hs4_codes)}개 품목, 계절성 제거 데이터 기준...")

for (col_a, col_b) in combinations(hs4_codes, 2):
    series_a = pivot_deseasoned[col_a]
    series_b = pivot_deseasoned[col_b]

    best_corr, best_lag = find_best_lag(series_a, series_b)

    # [수정] 'best_corr > MIN_CORR' -> 'abs(best_corr) > MIN_CORR'
    # (A -> B) 선후행 관계 (양/음 모두 포함)
    if MIN_LAG <= best_lag <= MAX_LAG and abs(best_corr) > MIN_CORR:
        rolling_corr = series_a.rolling(window=ROLLING_WINDOW).corr(series_b.shift(-best_lag))
        stability_std = rolling_corr.std()
        stability_mean = rolling_corr.mean()

        # [수정] 'stability_mean > 0.3' -> 'abs(stability_mean) > 0.3'
        if stability_std < MAX_STABILITY_STD and abs(stability_mean) > MIN_STABILITY_MEAN:
            comovement_pairs.append({
                'A_hs4': col_a, 'B_hs4': col_b, 'lag': best_lag,
                'corr': best_corr
            })

    # (B -> A) 선후행 관계 (양/음 모두 포함)
    elif MIN_LAG <= abs(best_lag) <= MAX_LAG and best_lag < 0 and abs(best_corr) > MIN_CORR:
        lag_ba = abs(best_lag)
        rolling_corr = series_b.rolling(window=ROLLING_WINDOW).corr(series_a.shift(-lag_ba))
        stability_std = rolling_corr.std()
        stability_mean = rolling_corr.mean()

        # [수정] 'stability_mean > 0.3' -> 'abs(stability_mean) > 0.3'
        if stability_std < MAX_STABILITY_STD and abs(stability_mean) > MIN_STABILITY_MEAN:
            comovement_pairs.append({
                'A_hs4': col_b, 'B_hs4': col_a, 'lag': best_lag,
                'corr': best_corr
            })

print(f"총 {len(comovement_pairs)}개의 유의미한 공행성 쌍을 찾았습니다.")

# 최종 공행성 쌍 DataFrame
if not comovement_pairs:
    print("경고: 공행성 쌍을 찾지 못했습니다. 기준을 완화해야 할 수 있습니다.")
    final_pairs_df = pd.DataFrame(columns=['A_hs4', 'B_hs4', 'lag'])
else:
    final_pairs_df = pd.DataFrame(comovement_pairs)
    final_pairs_df = final_pairs_df.drop_duplicates(subset=['A_hs4', 'B_hs4']) # 중복 쌍 제거


# # 9. 결과 정렬 및 HS코드명 매핑
# if comovement_pairs:
#     result_df = pd.DataFrame(comovement_pairs)

#     # [수정] 'corr' (상관계수)가 아닌 'abs_corr' (상관계수 절댓값) 기준으로 정렬
#     result_df['abs_corr'] = result_df['corr'].abs()
#     result_df = result_df.sort_values(by='abs_corr', ascending=False)

#     result_df['A_item'] = result_df['A_hs4'].map(hs4_map).fillna('알 수 없음')
#     result_df['B_item'] = result_df['B_hs4'].map(hs4_map).fillna('알 수 없음')

#     final_columns = ['A_hs4', 'A_item', 'B_hs4', 'B_item', 'lag', 'corr', 'abs_corr', 'stability_std', 'stability_mean']
#     print("\n--- 최종 공행성 후보 쌍 (계절성 제거, 안정성 강화, 음의 관계 포함) ---")
#     print(result_df[final_columns].head(20))
# else:
#     print("강화된 기준을 만족하는 공행성 쌍을 찾지 못했습니다.")

계절성 제거 완료.
탐색 시작: 51개 품목, 계절성 제거 데이터 기준...
총 31개의 유의미한 공행성 쌍을 찾았습니다.


2501  알 수 없음   2846  ...  0.710422       0.123153       -0.658756
12   2836  알 수 없음   7202  ...  0.697099       0.043201        0.719734
8    2805  알 수 없음   3006  ...  0.690503       0.110370       -0.763110
25   5205  알 수 없음   4810  ...  0.673691       0.092876       -0.724309
9    4202  알 수 없음   2805  ...  0.657540       0.074662       -0.654485
11   5705  알 수 없음   2811  ...  0.633861       0.132246       -0.678179
5    2833  알 수 없음   2529  ...  0.631167       0.110701       -0.672187
30   8708  알 수 없음   8527  ...  0.622707       0.143140        0.679999
3    2529  알 수 없음   2805  ...  0.609450       0.083343       -0.637231
29   8505  알 수 없음   8501  ...  0.607337       0.102437       -0.692087
7    2805  알 수 없음   2811  ...  0.606834       0.122633       -0.574691
21   3102  알 수 없음   6211  ...  0.598388       0.146057        0.638267
23   8527  알 수 없음   3806  ...  0.589027       0.145718        0.561710
17   2846  알 수 없음   3038  ...  0.579465       0.049433       -0.598620
28   8501  알 수 없음   7202  ...  0.572615       0.098768       -0.626328
0    1210  알 수 없음   5111  ...  0.572415       0.138183        0.555557
20   3038  알 수 없음   8708  ...  0.564967       0.117882        0.636661
14   8501  알 수 없음   2836  ...  0.562054       0.145168        0.491496
4    2811  알 수 없음   2529  ...  0.555918       0.136587       -0.546183
6    5402  알 수 없음   2529


1.소금, 순염화나트륨, 바닷물   -    세륨화합물(기타정밀화학원료)
2.탄산이나트륨(기타정밀화학원료) -   캣커트, 외과용, 접착제, 라미나리아.텐트, 봉합재,
3. 면사 ( 순면사) - 종이, 판지(필기용 인쇄용 그래픽용)  
4.트렁크 슈트케이스 화장품 케이스 등 가죽제가방 나트륨    ????
5.기타 양탄자, 바닥깔개   -    플루오르화수소 (기타정밀화학원료)
6. 황산이나트륨  -  장석(기타비금속광물)
7. 범퍼 완충기 그 부분품  -  라디오카세트플레이어
8. 장석 - 나트륨

2번은 탄산이나트륨은 유리 제조의 핵심 원료, 코로나 백신 접종 시기에 서로 영향을 많이 끼쳤던 물품들
6번은 인테리어 건설 붐에 같이 움직엿을 것
5번은 불산은 불소 화합물을 만드는 기초 원료. 그리고 이 불소 화합물은 양탄자의 얼룩방지 코팅에 수십년간 사용됨. 하지만 2021년부터 전 세계적으로 PFAS가 '영원한 화학물질'로 불리며 환경 규제 대상

핵심 전략 : Task2. final_pairs_df에 포함된 각 쌍(A, B)에 대해 개별 예측 모델을 만듭니다.이 모델은 A의 과거 데이터를 핵심 피처로 사용하여 B의 2025년 8월 값을 예측

In [ ]:
# ===2025sus 8월 예측 모델링(LGBM) ===
print("예측 모델 훈련 및 2025년 8월 값 예측 시작...")
def safe_get_value(pivot_df, date_key, col_key):
  if date_key in pivot_df.index:
    return pivot_df.loc[date_key, col_key]
  else:
    return 0.0

predictions = []
target_date = pd.to_datetime('2025-08-01')

# 예측할 쌍이 없는 경우 빈 리스트 처리
if final_pairs_df.empty:
    print("예측할 공행성 쌍이 없습니다.")
else:
    for _, row in final_pairs_df.iterrows():
        A_hs4 = row['A_hs4']
        B_hs4 = row['B_hs4']
        lag = int(row['lag'])

        # 1. 피처 및 타겟 생성
        # 예측 모델은 '원본 값(pivot_raw)'을 사용해야 함
        features = {}
        # 핵심 피처: A의 lag 적용 값
        features[f'A_lag_{lag}'] = pivot_raw[A_hs4].shift(lag)

        # 보조 피처: B의 자기회귀(AR) 피처 (최근 1, 12개월)
        features['B_lag_1'] = pivot_raw[B_hs4].shift(1)
        features['B_lag_12'] = pivot_raw[B_hs4].shift(12)

        # 시간 피처
        features['month'] = pivot_raw.index.month

        # 타겟
        features['target'] = pivot_raw[B_hs4]

        pair_train_df = pd.DataFrame(features).dropna()

        # 2. 모델 훈련
        if pair_train_df.empty:
            # 훈련 데이터가 부족하면 (lag가 너무 커서) 0으로 예측
            predicted_value = 0.0
        else:
            X_train = pair_train_df.drop(columns='target')
            y_train = pair_train_df['target']

            model = LGBMRegressor(random_state=42, n_estimators=100)
            model.fit(X_train, y_train)

            # 3. 2025년 8월 예측을 위한 피처 생성
            pred_features = {}

            # A의 lag 적용 값 (2025-08-01 기준)
            A_data_date = target_date - pd.DateOffset(months=lag)
            pred_features[f'A_lag_{lag}'] = safe_get_value(pivot_raw, A_data_date, A_hs4)

            B_lag_1_date = target_date - pd.DateOffset(months=1)
            B_lag_12_date = target_date - pd.DateOffset(months=12)

            # B의 lag 적용 값
            pred_features['B_lag_1'] = safe_get_value(pivot_raw, B_lag_1_date, B_hs4)
            pred_features['B_lag_12'] = safe_get_value(pivot_raw, B_lag_12_date, B_hs4)
            # 시간 피처
            pred_features['month'] = target_date.month

            X_pred = pd.DataFrame([pred_features])

            # 4. 예측
            predicted_value = model.predict(X_pred)[0]

        # 5. 결과 저장
        predictions.append({
            'leading_item_id': A_hs4,
            'following_item_id': B_hs4,
            'value': predicted_value
        })

# --- 4. 최종 제출 파일 생성 ---
print("제출 파일 생성 중...")

if not predictions:
    # 만약 예측할 쌍이 하나도 없었다면, 빈 DataFrame 생성
    submission_df = pd.DataFrame(columns=['leading_item_id', 'following_item_id', 'value'])
else:
    submission_df = pd.DataFrame(predictions)

    # 평가 산식에 맞춰 값 처리
    # 1. 음수 값은 0으로
    submission_df['value'] = submission_df['value'].apply(lambda x: max(0, x))
    # 2. 반올림 후 정수 변환
    submission_df['value'] = submission_df['value'].apply(lambda x: int(round(x)))

# 컬럼 순서 및 이름 확인 (평가 산식의 _validate_input 기준)
submission_df = submission_df[['leading_item_id', 'following_item_id', 'value']]

# 중복 쌍 확인 (평가 산식 기준)
if submission_df.duplicated(subset=['leading_item_id', 'following_item_id']).any():
    print("경고: 최종 제출 파일에 중복된 (선행, 후행) 쌍이 있습니다. 로직을 점검하세요.")
    # 중복 발생 시 첫 번째 값만 남김
    submission_df = submission_df.drop_duplicates(subset=['leading_item_id', 'following_item_id'], keep='first')

submission_df.to_csv('submission.csv', index=False)

print("--- submission.csv 파일 생성이 완료되었습니다. ---")
print(f"총 {len(submission_df)}개의 공행성 쌍을 제출합니다.")
print(submission_df.head())

예측 모델 훈련 및 2025년 8월 값 예측 시작...
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 31, number of used features: 0
[LightGBM] [Info] Start training from score 121823.161290
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there a

#Baseline 코드

##시차 상관관계를 이용한 브루트포스 탐색
모든 아이템 쌍에 대해 이중 반복문을 실행
min_nonzero=12 이상 데이터가 있는 아이템만 대상으로 필터링
1개월부터 6개월까지 시차를 1씩 늘려가며 A와 B의 상관관계 계산
1~6개월 시차 중 가장 높은 상관계수를 보인 시차를 best_lag로 선정
만약 이 best_corr(최고 상관계수)가 corr_threshold=0.4 이상이면 이 (A,B) 쌍을 공행성 쌍으로 채택



In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
train = pd.read_csv('./train.csv')

# year, month, item_id 기준으로 value 합산 (seq만 다르다면 value 합산)
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)

# year, month를 하나의 키(ym)로 묶기
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)

# item_id × ym 피벗 (월별 총 무역량 매트릭스 생성)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)

pivot.head()
def safe_corr(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])

def find_comovement_pairs(pivot, max_lag=6, min_nonzero=12, corr_threshold=0.4):
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)

    results = []

    for i, leader in tqdm(enumerate(items)):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        for follower in items:
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0

            # lag = 1 ~ max_lag 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue
                corr = safe_corr(x[:-lag], y[lag:])
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            # 임계값 이상이면 공행성쌍으로 채택
            if best_lag is not None and abs(best_corr) >= corr_threshold:
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                })

    pairs = pd.DataFrame(results)
    return pairs

pairs = find_comovement_pairs(pivot)
print("탐색된 공행성쌍 수:", len(pairs))
pairs.head()
def build_training_data(pivot, pairs):
    """
    공행성쌍 + 시계열을 이용해 (X, y) 학습 데이터를 만드는 함수
    input X:
      - b_t, b_t_1, a_t_lag, max_corr, best_lag
    target y:
      - b_t_plus_1
    """
    months = pivot.columns.to_list()
    n_months = len(months)

    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t+1이 존재하고, t-lag >= 0인 구간만 학습에 사용
        for t in range(max(lag, 1), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag]
            b_t_plus_1 = b_series[t + 1]

            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
                "target": b_t_plus_1,
            })

    df_train = pd.DataFrame(rows)
    return df_train

df_train_model = build_training_data(pivot, pairs)
print('생성된 학습 데이터의 shape :', df_train_model.shape)
df_train_model.head()
# 회귀모델 학습
feature_cols = ['b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag']

train_X = df_train_model[feature_cols].values
train_y = df_train_model["target"].values

reg = LinearRegression()
reg.fit(train_X, train_y)
def predict(pivot, pairs, reg):
    months = pivot.columns.to_list()
    n_months = len(months)

    # 가장 마지막 두 달 index (2025-7, 2025-6)
    t_last = n_months - 1
    t_prev = n_months - 2

    preds = []

    for row in tqdm(pairs.itertuples(index=False)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # t_last - lag 가 0 이상인 경우만 예측
        if t_last - lag < 0:
            continue

        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev]
        a_t_lag = a_series[t_last - lag]

        X_test = np.array([[b_t, b_t_1, a_t_lag, corr, float(lag)]])
        y_pred = reg.predict(X_test)[0]

        # (후처리 1) 음수 예측 → 0으로 변환
        # (후처리 2) 소수점 → 정수 변환 (무역량은 정수 단위)
        y_pred = max(0.0, float(y_pred))
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })

    df_pred = pd.DataFrame(preds)
    return df_pred
submission = predict(pivot, pairs, reg)
print(submission.head())
submission.to_csv('./baseline_submit.csv', index=False)

100it [00:11,  8.97it/s]


탐색된 공행성쌍 수: 1425
생성된 학습 데이터의 shape : (54743, 6)


1425it [00:00, 4456.06it/s]

  leading_item_id following_item_id    value
0        AANGBULD          APQGTRMF   360075
1        AANGBULD          DEWLVASR   610115
2        AANGBULD          DNMPSKTB  4999179
3        AANGBULD          EVBVXETX  4904136
4        AANGBULD          FTSVTTSR   387062


여기서 추가로 해볼 수 있는 거 세 가지
1. 예측 모델 변경 (선형 회귀 -> 트리기반 부스팅 모델)
LGMRegressor 또는 XGBRegressor를 사용
2. 피처 엔지니어링
시간/계절성 피처 추가 : 무역 데이터는 계절성이 매우 강함
df_train_model['month'] = (t + 1) % 12
롤링 피처 추가 : B의 최근 추세를 모델에 알려줌
b_t의 3개월/6개월 롤링 평균, 롤링 표준편차
A의 추가 Lag 피처 : a_t_lag 외에도 a_t_lag-1, a_t_lag + 1등 주변 시점의 데이터 추가
B의 추가 Lag 피처 : b_t_1 외에도 b_t_2, b_t_12등을 추가

3. 원본 값을 보지 말고 움직임을 볼 수도 있음
pivot데이터를 가공
로그 변환: pivot_log = np.log1p(pivot) (값의 편차를 줄여줌)

차분(Differencing): pivot_diff = pivot_log.diff(axis=1).dropna(axis=1) (추세를 제거하고 변동량만 봄)

가공된 pivot_diff 데이터로 상관관계를 계산하면, 단순히 값이 같이 높은 쌍이 아니라 "같이 증가/감소하는" 쌍을 찾을 수 있습니다.

#1번째 시도 - LightGBM 모델 적용

In [ ]:
import pandas as pd
import numpy as np
# [수정] LinearRegression 대신 LGBMRegressor 임포트
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from tqdm import tqdm

train = pd.read_csv('./train.csv')

# year, month, item_id 기준으로 value 합산 (seq만 다르다면 value 합산)
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)

# year, month를 하나의 키(ym)로 묶기
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)

# item_id × ym 피벗 (월별 총 무역량 매트릭스 생성)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)

# --- (1. 공행성 쌍 탐색 - 변경 없음) ---
def safe_corr(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])

def find_comovement_pairs(pivot, max_lag=6, min_nonzero=12, corr_threshold=0.4):
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)
    results = []

    for i, leader in tqdm(enumerate(items)):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue
        for follower in items:
            if follower == leader:
                continue
            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue
                corr = safe_corr(x[:-lag], y[lag:])
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            if best_lag is not None and abs(best_corr) >= corr_threshold:
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                })
    pairs = pd.DataFrame(results)
    return pairs

pairs = find_comovement_pairs(pivot)
print("탐색된 공행성쌍 수:", len(pairs))


# --- (2. 학습 데이터 구축 - 변경 없음) ---
def build_training_data(pivot, pairs):
    months = pivot.columns.to_list()
    n_months = len(months)
    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        if leader not in pivot.index or follower not in pivot.index:
            continue
        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        for t in range(max(lag, 1), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag]
            b_t_plus_1 = b_series[t + 1]
            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
                "target": b_t_plus_1,
            })
    df_train = pd.DataFrame(rows)
    return df_train

df_train_model = build_training_data(pivot, pairs)
print('생성된 학습 데이터의 shape :', df_train_model.shape)


# --- (3. 회귀모델 학습 - [수정됨]) ---
feature_cols = ['b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag']

train_X = df_train_model[feature_cols].values
train_y = df_train_model["target"].values

# [수정] 모델을 LinearRegression -> LGBMRegressor로 변경
# reg = LinearRegression()
reg = LGBMRegressor(random_state=42, n_estimators=100) # n_estimators는 트리의 개수

print("LGBMRegressor 모델 학습 시작...")
reg.fit(train_X, train_y)
print("모델 학습 완료.")


# --- (4. 예측 - 변경 없음) ---
def predict(pivot, pairs, reg):
    months = pivot.columns.to_list()
    n_months = len(months)
    t_last = n_months - 1
    t_prev = n_months - 2
    preds = []

    for row in tqdm(pairs.itertuples(index=False)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        if leader not in pivot.index or follower not in pivot.index:
            continue
        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)
        if t_last - lag < 0:
            continue

        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev]
        a_t_lag = a_series[t_last - lag]

        X_test = np.array([[b_t, b_t_1, a_t_lag, corr, float(lag)]])
        y_pred = reg.predict(X_test)[0]

        y_pred = max(0.0, float(y_pred))
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })
    df_pred = pd.DataFrame(preds)
    return df_pred

submission = predict(pivot, pairs, reg)
submission.to_csv('./lgbm_submit.csv', index=False) # 파일 이름 변경

print("LGBM 모델 예측 완료. lgbm_submit.csv 저장됨.")
print(submission.head())

100it [00:05, 19.22it/s]


탐색된 공행성쌍 수: 2081
생성된 학습 데이터의 shape : (83214, 6)
LGBMRegressor 모델 학습 시작...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001566 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1024
[LightGBM] [Info] Number of data points in the train set: 83214, number of used features: 5
[LightGBM] [Info] Start training from score 4938983.363821
모델 학습 완료.


2081it [00:02, 1002.93it/s]

LGBM 모델 예측 완료. lgbm_submit.csv 저장됨.
  leading_item_id following_item_id    value
0        AANGBULD          BEZYMBBT  3158507
1        AANGBULD          DDEXPPXU    59208
2        AANGBULD          DEWLVASR   409895
3        AANGBULD          DNMPSKTB  5543362
4        AANGBULD          EVBVXETX  4843997


#LightGBM 모델을 썼을 때 정확도가 0.3200737486에서 0.3232141477로 상승
비교해보고자 한것 원본의 값이 나은지 움직임을 보는게 나은지
베이스라인은 원본(raw) 값의 상관관계를 봅니다. 하지만 이전 질문에서 시도하셨던 것처럼, 원본 값보다는 **변동성(움직임)**을 보는 것이 더 정확할 수 있습니다.

find_comovement_pairs 함수에 전달하는 pivot 데이터를 가공합니다.

로그 변환: pivot_log = np.log1p(pivot) (값의 편차를 줄여줌)

차분(Differencing): pivot_diff = pivot_log.diff(axis=1).dropna(axis=1) (추세를 제거하고 변동량만 봄)

가공된 pivot_diff 데이터로 상관관계를 계산하면, 단순히 값이 같이 높은 쌍이 아니라 "같이 증가/감소하는" 쌍을 찾을 수 있습니다.
이걸 적용했을 때의 코드
의문 ? 값이 같이 높고 낮은 것을 봐야하는지 아니면 같이 움직이는 애들을 봐야하는지


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from tqdm import tqdm

train = pd.read_csv('./train.csv')

# --- 데이터 전처리 (기존과 동일) ---
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)

# --- [수정] 공행성 탐색용 '변동성' 데이터 생성 ---
# 1. 로그 변환: 값의 편차를 줄여줌 (0 처리를 위해 log1p 사용)
pivot_log = np.log1p(pivot)

# 2. 차분(Differencing): 추세를 제거하고 '변동량(움직임)'만 봄
# axis=1 (시간 방향)으로 차분을 계산
pivot_diff = pivot_log.diff(axis=1)

# 3. 차분으로 인해 첫 번째 달(2022-01)이 NaN이 되므로 해당 열(column)을 제거
pivot_diff = pivot_diff.dropna(axis=1)

print("공행성 탐색용 데이터(로그+차분) 생성 완료.")
print(f"원본 pivot shape: {pivot.shape}")
print(f"변환 pivot_diff shape: {pivot_diff.shape}")


# --- (1. 공행성 쌍 탐색 - 함수 정의는 변경 없음) ---
def safe_corr(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])

def find_comovement_pairs(pivot, max_lag=6, min_nonzero=12, corr_threshold=0.4):
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)
    results = []

    # [참고] pivot_diff는 0이 많을 수 있으므로 min_nonzero 기준이 다르게 동작할 수 있음
    for i, leader in tqdm(enumerate(items)):
        # pivot.loc 대신 pivot_diff.loc를 사용하기 위해 item이 pivot_diff에 있는지 확인
        if leader not in pivot.index: continue

        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        for follower in items:
            if follower == leader: continue
            if follower not in pivot.index: continue

            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue
                # [수정] 상관관계 계산 시 원본 x, y가 아닌 변환된 series 사용
                # x_diff = pivot.loc[leader].values[1:] # pivot_diff의 값들
                # y_diff = pivot.loc[follower].values[1:] # pivot_diff의 값들

                # [수정] pivot에서 직접 시리즈를 가져와서 상관관계 계산
                x_series_diff = pivot.loc[leader].values
                y_series_diff = pivot.loc[follower].values

                # [수정] pivot_diff에서 시리즈를 가져오도록 수정
                if leader not in pivot_diff.index or follower not in pivot_diff.index:
                    continue

                x_series_diff = pivot_diff.loc[leader].values
                y_series_diff = pivot_diff.loc[follower].values

                # n_months는 pivot_diff의 컬럼 수가 되어야 함
                n_months_diff = len(x_series_diff)
                if n_months_diff <= lag:
                    continue

                corr = safe_corr(x_series_diff[:-lag], y_series_diff[lag:])

                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            if best_lag is not None and abs(best_corr) >= corr_threshold:
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                })
    pairs = pd.DataFrame(results)
    return pairs

# [수정] find_comovement_pairs에 원본 pivot 대신 pivot_diff를 전달
print("--- '변동성' 기반 공행성 쌍 탐색 시작 ---")
pairs = find_comovement_pairs(pivot_diff, corr_threshold=0.3) # 변동성은 상관계수가 낮게 나오므로 임계값 조정 (예: 0.3)
print("탐색된 공행성쌍 수:", len(pairs))


# --- (2. 학습 데이터 구축 - [중요] 원본 pivot 사용, 변경 없음) ---
def build_training_data(pivot, pairs):
    months = pivot.columns.to_list()
    n_months = len(months)
    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        # [중요] pivot에 item이 있는지 확인 (원본 pivot 기준)
        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        for t in range(max(lag, 1), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag]
            b_t_plus_1 = b_series[t + 1]
            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
                "target": b_t_plus_1,
            })
    df_train = pd.DataFrame(rows)
    return df_train

# [중요] 학습 데이터 생성 시 '원본' pivot을 전달
print("--- '원본 값' 기준 학습 데이터 구축 시작 ---")
df_train_model = build_training_data(pivot, pairs)
print('생성된 학습 데이터의 shape :', df_train_model.shape)


# --- (3. 회귀모델 학습 - LGBM 사용, 변경 없음) ---
feature_cols = ['b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag']

# 학습 데이터가 없는 경우 예외 처리
if df_train_model.empty:
    print("오류: 학습 데이터가 없습니다. 공행성 쌍 탐색 기준(corr_threshold)을 확인하세요.")
    # 빈 submission 파일 생성
    submission = pd.DataFrame(columns=['leading_item_id', 'following_item_id', 'value'])
else:
    train_X = df_train_model[feature_cols].values
    train_y = df_train_model["target"].values

    reg = LGBMRegressor(random_state=42, n_estimators=100)
    print("LGBMRegressor 모델 학습 시작...")
    reg.fit(train_X, train_y)
    print("모델 학습 완료.")

# --- (4. 예측 - [중요] 원본 pivot 사용, 변경 없음) ---
def predict(pivot, pairs, reg):
    months = pivot.columns.to_list()
    n_months = len(months)
    t_last = n_months - 1
    t_prev = n_months - 2
    preds = []

    for row in tqdm(pairs.itertuples(index=False)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        # [중요] pivot에 item이 있는지 확인 (원본 pivot 기준)
        if leader not in pivot.index or follower not in pivot.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        if t_last - lag < 0:
            continue

        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev]
        a_t_lag = a_series[t_last - lag]

        X_test = np.array([[b_t, b_t_1, a_t_lag, corr, float(lag)]])
        y_pred = reg.predict(X_test)[0]

        y_pred = max(0.0, float(y_pred))
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })
    df_pred = pd.DataFrame(preds)
    return df_pred

# [중V] 예측 시 '원본' pivot과 학습된 reg 전달
if df_train_model.empty:
    print("학습된 모델이 없어 예측을 건너뜁니다.")
else:
    print("--- '원본 값' 기준 예측 시작 ---")
    submission = predict(pivot, pairs, reg)
    print(submission.head())

submission.to_csv('./lgbm_diff_submit.csv', index=False) # 파일 이름 변경
print("LGBM(변동성 기반) 모델 예측 완료. lgbm_diff_submit.csv 저장됨.")

공행성 탐색용 데이터(로그+차분) 생성 완료.
원본 pivot shape: (100, 43)
변환 pivot_diff shape: (100, 42)
--- '변동성' 기반 공행성 쌍 탐색 시작 ---


100it [00:19,  5.20it/s]


탐색된 공행성쌍 수: 4061
--- '원본 값' 기준 학습 데이터 구축 시작 ---
생성된 학습 데이터의 shape : (155858, 6)
LGBMRegressor 모델 학습 시작...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002891 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1027
[LightGBM] [Info] Number of data points in the train set: 155858, number of used features: 5
[LightGBM] [Info] Start training from score 4224180.653454
모델 학습 완료.
--- '원본 값' 기준 예측 시작 ---


4061it [00:03, 1069.21it/s]

  leading_item_id following_item_id    value
0        AANGBULD          BEZYMBBT  3078786
1        AANGBULD          BJALXPFS   170196
2        AANGBULD          BLANHGYY    57501
3        AANGBULD          BTMOEMEP  7964926
4        AANGBULD          DDEXPPXU    44190
LGBM(변동성 기반) 모델 예측 완료. lgbm_diff_submit.csv 저장됨.


정확도가 0.19868로 매우 낮아짐 -> 변동성을 기준으로 보는 게 아니라 원본값을 기준으로 보는게 맞는 것 같음
주최측에서 원본값을 보는 걸로 결과값이 나온 듯
->그럼 여기서 또해볼 수 있는 시도는 뭐가 있을까?
1. S2 개선 : 예측 모델 피처 엔지니어링
액션: build_training_data 함수에 계절성과 과거 추세 피처를 추가합니다.
month (월): 무역 데이터에서 가장 중요한 계절성 피처입니다.
b_t_12 (1년 전 B의 값): 1년 전 동월의 값은 다음 달 예측에 강력한 힌트가 됩니다.
b_roll_mean_3 (B의 3개월 이동평균): B의 최근 추세를 반영합니다.
이 피처들을 추가하여 모델(LGBMRegressor)이 더 정확하게 예측하도록 만듭니다.
2. 공행성 쌍 탐색 파라미터 튜닝
s1 점수는 find_comovement_pairs 함수에 전적으로 달려있음
corr_threshold=0.4: 이 임계값이 F1 점수를 결정하는 핵심입니다.
0.4가 너무 높으면: 정답(G)을 너무 적게 찾아내(Recall 하락) F1이 낮아집니다.
0.4가 너무 낮으면: 오답(FP)을 너무 많이 찾아내(Precision 하락) F1이 낮아집니다.
액션: corr_threshold 값을 0.3, 0.35, 0.45, 0.5 등으로 변경해 보면서 어떤 값이 가장 높은 점수(0.3 이상)를 주는지 **실험(Tuning)**해야 합니다.

#피처 엔지니어링 추가, threshold값은 0.3으로 하기

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from tqdm import tqdm

train = pd.read_csv('./train.csv')

# --- 데이터 전처리 (변경 없음) ---
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)
months_dt = pivot.columns.to_list() # 월(datetime) 리스트

# --- (1. 공행성 쌍 탐색 - 원본 pivot 사용) ---
def safe_corr(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])

def find_comovement_pairs(pivot, max_lag=6, min_nonzero=12, corr_threshold=0.3):
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)
    results = []

    for i, leader in tqdm(enumerate(items)):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue
        for follower in items:
            if follower == leader:
                continue
            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue
                corr = safe_corr(x[:-lag], y[lag:])
                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag

            if best_lag is not None and abs(best_corr) >= corr_threshold:
                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr": best_corr,
                })
    pairs = pd.DataFrame(results)
    return pairs

# [중요] 원본 pivot을 사용. S1(F1) 점수를 위해 corr_threshold를 튜닝해야 함.
print("--- '원본 값' 기반 공행성 쌍 탐색 시작 ---")
pairs = find_comovement_pairs(pivot, corr_threshold=0.3)
print("탐색된 공행성쌍 수:", len(pairs))


# --- (2. 학습 데이터 구축 - [수정됨] 피처 엔지니어링) ---
def build_training_data(pivot, pairs, months_dt):
    months = months_dt
    n_months = len(months)
    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        if leader not in pivot.index or follower not in pivot.index:
            continue
        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # [수정] 1년(12개월)치 피처를 사용하기 위해 t의 시작점을 12부터로 변경
        for t in range(max(lag, 12), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag]
            b_t_plus_1 = b_series[t + 1]

            # [새 피처 1] B의 1년 전 동월 값
            b_t_12 = b_series[t - 12]

            # [새 피처 2] B의 3개월 롤링 평균
            b_roll_mean_3 = (b_series[t] + b_series[t-1] + b_series[t-2]) / 3

            # [새 피처 3] 예측할 달 (t+1)의 월(month) 정보 (계절성)
            target_month = months[t + 1].month

            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
                "b_t_12": b_t_12,             # <-- 추가
                "b_roll_mean_3": b_roll_mean_3, # <-- 추가
                "month": float(target_month), # <-- 추가
                "target": b_t_plus_1,
            })
    df_train = pd.DataFrame(rows)
    return df_train

df_train_model = build_training_data(pivot, pairs, months_dt)
print('생성된 학습 데이터의 shape :', df_train_model.shape)


# --- (3. 회귀모델 학습 - [수정됨] 피처 리스트) ---
# [수정] 5개 -> 8개 피처로 변경
feature_cols = [
    'b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag',
    'b_t_12', 'b_roll_mean_3', 'month' # <-- 추가
]

if df_train_model.empty:
    print("오류: 학습 데이터가 없습니다. (lag가 너무 크거나, t 시작점이 높을 수 있음)")
    submission = pd.DataFrame(columns=['leading_item_id', 'following_item_id', 'value'])
else:
    train_X = df_train_model[feature_cols].values
    train_y = df_train_model["target"].values

    reg = LGBMRegressor(random_state=42, n_estimators=500, learning_rate= 0.01)
    print("LGBMRegressor 모델(피처 추가) 학습 시작...")
    reg.fit(train_X, train_y)
    print("모델 학습 완료.")


# --- (4. 예측 - [수정됨] 피처 생성) ---
def predict(pivot, pairs, reg, months_dt):
    months = months_dt
    n_months = len(months)
    t_last = n_months - 1
    t_prev = n_months - 2
    preds = []

    # 예측할 달(2025년 8월)의 '월' 정보
    target_month = months[-1].month + 1 if months[-1].month < 12 else 1

    for row in tqdm(pairs.itertuples(index=False)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)
        if leader not in pivot.index or follower not in pivot.index:
            continue
        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        # [수정] 12개월치 피처를 쓰기 위해 t_last가 12보다 큰지 확인
        if t_last - lag < 0 or t_last < 12:
            continue

        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev]
        a_t_lag = a_series[t_last - lag]

        # [새 피처]
        b_t_12 = b_series[t_last - 12]
        b_roll_mean_3 = (b_series[t_last] + b_series[t_last-1] + b_series[t_last-2]) / 3

        # [수정] 8개 피처로 X_test 구성
        X_test = np.array([[
            b_t, b_t_1, a_t_lag, corr, float(lag),
            b_t_12, b_roll_mean_3, float(target_month) # <-- 추가
        ]])

        y_pred = reg.predict(X_test)[0]

        y_pred = max(0.0, float(y_pred))
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })
    df_pred = pd.DataFrame(preds)
    return df_pred

if df_train_model.empty:
    print("학습된 모델이 없어 예측을 건너뜁니다.")
else:
    print("--- '피처 추가' 모델 예측 시작 ---")
    submission = predict(pivot, pairs, reg, months_dt)
    print(submission.head())

submission.to_csv('./lgbm_raw_featured_submit.csv', index=False)
print("LGBM(원본, 피처추가) 모델 예측 완료. lgbm_raw_featured_submit.csv 저장됨.")

--- '원본 값' 기반 공행성 쌍 탐색 시작 ---


100it [00:10,  9.45it/s]


탐색된 공행성쌍 수: 3242
생성된 학습 데이터의 shape : (97260, 9)
LGBMRegressor 모델(피처 추가) 학습 시작...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002700 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1550
[LightGBM] [Info] Number of data points in the train set: 97260, number of used features: 8
[LightGBM] [Info] Start training from score 4165565.445096
모델 학습 완료.
--- '피처 추가' 모델 예측 시작 ---


3242it [00:03, 828.07it/s]

  leading_item_id following_item_id    value
0        AANGBULD          APQGTRMF   129661
1        AANGBULD          BEZYMBBT  3885749
2        AANGBULD          DDEXPPXU   129661
3        AANGBULD          DEWLVASR   333263
4        AANGBULD          DNMPSKTB  5321295
LGBM(원본, 피처추가) 모델 예측 완료. lgbm_raw_featured_submit.csv 저장됨.


정확도가 0.33401로 상승함 -> 이걸 통해 알 수 있는 점
피처를 좀 더 추가하면 정확도가 상승함 corr변수를 0.4보단 0.3이 더 정확함, 0.35보다도 0.3이 더 정확한 듯 LGBM모델을 쓰는 건 맞음, 변동성보다는 원본값을 보는 게 더 정확도가 높음
그럼 이제 여기서 더 어떻게 해야할까를 고민해봐야 할 듯

그 다음올 해볼 수 있는 것
1. 관계의 안정성으로 쌍 필터링
F1 점수가 낮다는 것은 정확도가 낮다는 의미일 수 있음. 즉 우연히 상관관계가 높아서 가짜 쌍을 너무 많이 제출하고 있을 수 있음
방법: find_comovement_pairs 함수에서 best_corr >= corr_threshold 조건에 '롤링 상관계수의 표준편차' 조건을 추가합니다. (사용자님이 초기에 작성했던 코드와 유사)

왜 효과가 있나요?

전체 기간의 상관관계(best_corr)는 높지만, 특정 기간에만 상관관계가 높고 나머지는 낮다면 '가짜' 관계일 수 있습니다.

롤링 상관계수의 표준편차(rolling_corr.std())가 낮다는 것은 **"두 관계가 꾸준하고 안정적"**이라는 의미입니다.

이 필터링을 통해 '가짜 쌍'(FP)을 줄여 S1(F1) 점수의 Precision을 높일 수 있습니다.

적용한 점 일단 모델의 파라미터를 넣어줌 estimators를 좀 더 높여주고 learning_ate를 0.01로 함, 안정성과 변동성 상관관계를 필터링하지 않고 추가정보로 제공하기
스스로 학습하게 하기


In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from tqdm import tqdm

train = pd.read_csv('./train.csv')

# --- 데이터 전처리 (변경 없음) ---
monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)["value"]
    .sum()
)
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)
pivot = (
    monthly
    .pivot(index="item_id", columns="ym", values="value")
    .fillna(0.0)
)
months_dt = pivot.columns.to_list()

# [전략 1] '변동성' 데이터도 미리 생성
pivot_log = np.log1p(pivot)
pivot_diff = pivot_log.diff(axis=1).dropna(axis=1)


# --- (1. 공행성 쌍 탐색 - [수정] 피처 추가) ---
def safe_corr(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0
    return float(np.corrcoef(x, y)[0, 1])

def find_comovement_pairs(pivot, pivot_diff, max_lag=6, min_nonzero=12, corr_threshold=0.3):
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)
    results = []

    for i, leader in tqdm(enumerate(items)):
        if leader not in pivot.index or leader not in pivot_diff.index: continue

        x_raw = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x_raw) < min_nonzero:
            continue

        x_diff = pivot_diff.loc[leader].values.astype(float) # 변동성 X

        for follower in items:
            if follower == leader: continue
            if follower not in pivot.index or follower not in pivot_diff.index: continue

            y_raw = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y_raw) < min_nonzero:
                continue

            y_diff = pivot_diff.loc[follower].values.astype(float) # 변동성 Y

            best_lag = None
            best_corr_raw = 0.0

            # 1. 원본 값(raw) 기준으로 최적 lag, corr 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag: continue
                corr = safe_corr(x_raw[:-lag], y_raw[lag:])
                if abs(corr) > abs(best_corr_raw):
                    best_corr_raw = corr
                    best_lag = lag

            # 2. '원본 값' 상관관계가 임계값을 넘으면 (필터링 조건은 S1을 위해 유지)
            if best_lag is not None and abs(best_corr_raw) >= corr_threshold:

                # 3. [추가 피처] '변동성 상관관계'와 '안정성' 계산

                # 변동성 상관관계 (차분 데이터는 길이가 1 짧음)
                n_months_diff = len(x_diff)
                corr_diff = 0.0
                if n_months_diff > best_lag:
                    corr_diff = safe_corr(x_diff[:-best_lag], y_diff[best_lag:])

                # 안정성 (Rolling Std)
                s_x = pd.Series(x_raw[:-best_lag])
                s_y = pd.Series(y_raw[best_lag:])
                rolling_corr = s_x.rolling(window=12).corr(s_y)
                stability_std = rolling_corr.std()

                # NaN이면 0.0으로 대체 (계산 불능 시)
                stability_std = stability_std if pd.notna(stability_std) else 0.0

                results.append({
                    "leading_item_id": leader,
                    "following_item_id": follower,
                    "best_lag": best_lag,
                    "max_corr_raw": best_corr_raw,     # <-- 피처 1 (기존)
                    "max_corr_diff": corr_diff,   # <-- 피처 2 (추가)
                    "stability_std": stability_std  # <-- 피처 3 (추가)
                })
    pairs = pd.DataFrame(results)
    return pairs

# [수정] pivot와 pivot_diff 둘 다 전달
print("--- '하이브리드 피처' 기반 공행성 쌍 탐색 시작 ---")
pairs = find_comovement_pairs(pivot, pivot_diff, corr_threshold=0.3)
print("탐색된 공행성쌍 수:", len(pairs))


# --- (2. 학습 데이터 구축 - [수정] 새 피처 반영) ---
def build_training_data(pivot, pairs, months_dt):
    months = months_dt
    n_months = len(months)
    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)

        # [수정] 3개의 피처를 가져옴
        corr_raw = float(row.max_corr_raw)
        corr_diff = float(row.max_corr_diff)
        stability = float(row.stability_std)

        if leader not in pivot.index or follower not in pivot.index:
            continue
        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        for t in range(max(lag, 12), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            a_t_lag = a_series[t - lag]
            b_t_plus_1 = b_series[t + 1]
            b_t_12 = b_series[t - 12]
            b_roll_mean_3 = (b_series[t] + b_series[t-1] + b_series[t-2]) / 3
            target_month = months[t + 1].month

            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "a_t_lag": a_t_lag,
                "best_lag": float(lag),
                "b_t_12": b_t_12,
                "b_roll_mean_3": b_roll_mean_3,
                "month": float(target_month),
                "max_corr_raw": corr_raw,       # <-- 추가
                "max_corr_diff": corr_diff,     # <-- 추가
                "stability_std": stability,     # <-- 추가
                "target": b_t_plus_1,
            })
    df_train = pd.DataFrame(rows)
    return df_train

df_train_model = build_training_data(pivot, pairs, months_dt)
print('생성된 학습 데이터의 shape :', df_train_model.shape)


# --- (3. 회귀모델 학습 - [수정] 피처 리스트) ---
# [수정] 8개 -> 10개 피처로 변경
feature_cols = [
    'b_t', 'b_t_1', 'a_t_lag', 'best_lag',
    'b_t_12', 'b_roll_mean_3', 'month',
    'max_corr_raw', 'max_corr_diff', 'stability_std' # <-- 추가
]

if df_train_model.empty:
    print("오류: 학습 데이터가 없습니다.")
    submission = pd.DataFrame(columns=['leading_item_id', 'following_item_id', 'value'])
else:
    train_X = df_train_model[feature_cols].values
    train_y = df_train_model["target"].values

    # [수정] 튜닝된 파라미터 적용 (예시)
    reg = LGBMRegressor(random_state=42, n_estimators=500, learning_rate=0.01)
    print("LGBMRegressor 모델(하이브리드 피처) 학습 시작...")
    reg.fit(train_X, train_y)
    print("모델 학습 완료.")


# --- (4. 예측 - [수정] 피처 생성) ---
def predict(pivot, pairs, reg, months_dt):
    months = months_dt
    n_months = len(months)
    t_last = n_months - 1
    t_prev = n_months - 2
    preds = []

    target_month = months[-1].month + 1 if months[-1].month < 12 else 1

    for row in tqdm(pairs.itertuples(index=False)):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)

        # [수정] 3개의 피처를 가져옴
        corr_raw = float(row.max_corr_raw)
        corr_diff = float(row.max_corr_diff)
        stability = float(row.stability_std)

        if leader not in pivot.index or follower not in pivot.index:
            continue
        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)

        if t_last - lag < 0 or t_last < 12:
            continue

        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev]
        a_t_lag = a_series[t_last - lag]

        b_t_12 = b_series[t_last - 12]
        b_roll_mean_3 = (b_series[t_last] + b_series[t_last-1] + b_series[t_last-2]) / 3

        # [수정] 10개 피처로 X_test 구성
        X_test = np.array([[
            b_t, b_t_1, a_t_lag, float(lag),
            b_t_12, b_roll_mean_3, float(target_month),
            corr_raw, corr_diff, stability # <-- 추가
        ]])

        y_pred = reg.predict(X_test)[0]
        y_pred = max(0.0, float(y_pred))
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })
    df_pred = pd.DataFrame(preds)
    return df_pred

if df_train_model.empty:
    print("학습된 모델이 없어 예측을 건너뜁니다.")
else:
    print("--- '하이브리드 피처' 모델 예측 시작 ---")
    submission = predict(pivot, pairs, reg, months_dt)
    print(submission.head())

submission.to_csv('./lgbm_hybrid_featured_submit.csv', index=False)
print("LGBM(하이브리드) 모델 예측 완료. lgbm_hybrid_featured_submit.csv 저장됨.")

--- '하이브리드 피처' 기반 공행성 쌍 탐색 시작 ---


100it [00:12,  7.78it/s]


탐색된 공행성쌍 수: 3242
생성된 학습 데이터의 shape : (97260, 11)
LGBMRegressor 모델(하이브리드 피처) 학습 시작...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003296 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2060
[LightGBM] [Info] Number of data points in the train set: 97260, number of used features: 10
[LightGBM] [Info] Start training from score 4165565.445096
모델 학습 완료.
--- '하이브리드 피처' 모델 예측 시작 ---


3242it [00:03, 835.31it/s]


  leading_item_id following_item_id    value
0        AANGBULD          APQGTRMF   129661
1        AANGBULD          BEZYMBBT  3885749
2        AANGBULD          DDEXPPXU   129661
3        AANGBULD          DEWLVASR   333263
4        AANGBULD          DNMPSKTB  5321295
LGBM(하이브리드) 모델 예측 완료. lgbm_hybrid_featured_submit.csv 저장됨.


지금까지의 학습을 통해 깨달은점 : 0.3이 제일 나은 듯 (그렇다면 0.25로 더 내리지 않고 0.35로 하지 않아도 되나?) -> 이거 테스트 해보고 싶음
